## install staf

In [1]:
import warnings
warnings.filterwarnings('ignore')
from pinecone import Pinecone
from tqdm.auto import tqdm
from DLAIUtils import Utils
from PyPDF2 import PdfReader

import pandas as pd
utils = Utils()
PINECONE_API_KEY = utils.get_pinecone_api_key()

In [2]:
utils = Utils()
INDEX_NAME = 'health'


## build the index

In [7]:
import tiktoken
import langchain

tiktoken.encoding_for_model('gpt-3.5-turbo')

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

tiktoken_len("hello I am a chunk of text and using the tiktoken_len function "
             "we can find the length of this chunk of text in tokens")

26

In [15]:
book_dict = {'Outlive':r"C:\Users\yotam\Desktop\rag_p\books\Outlive.pdf",
             'The_4_Hour_Body':r"C:\Users\yotam\Desktop\rag_p\books\The_4_Hour_Body.pdf",
             'How_Not_to_Die':r"C:\Users\yotam\Desktop\rag_p\books\How_Not_to_Die.pdf"}
books_names = ['Outlive','The_4_Hour_Body','How_Not_to_Die']
book_list =[]
for book in book_dict:
    book_list.append(PdfReader(book_dict[book]))


In [22]:
full_books = []
page_dicts_list = []
for book in book_list:
    full_book = ''
    page_dict = {} #num:(start,end,lenght)
    for i in range(len(book.pages)):
        page = book.pages[i].extract_text().replace('\t',' ')#.replace('\n',' ')
        lenght = tiktoken_len(page)
        start = tiktoken_len(full_book)+1
        full_book = full_book + page + ' '
        end = tiktoken_len(full_book)
        page_dict[i] = (start,end,lenght)
    full_books.append(full_book)
    page_dicts_list.append(page_dict)
    print(tiktoken_len(full_book))

269821
244413
397940


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=75,
    length_function=tiktoken_len#,
    #separators=["\n\n", "\n", " ", ""]
)

In [8]:
from sentence_transformers import SentenceTransformer

model_emmbed = SentenceTransformer('BAAI/bge-large-en-v1.5')
def get_embeddings_vector(txt):
    #model = SentenceTransformer('BAAI/bge-large-en-v1.5')
    return model_emmbed.encode(txt)


In [32]:
txt_chunks = text_splitter.split_text(full_books[2])

In [33]:
len(txt_chunks)

1263

In [9]:
pinecone = Pinecone(api_key='ENTER YOUR PINCONE API KEY, HERE OR IN THE ENV')

index = pinecone.Index('health-bge-large')
#index.describe_index_stats()

In [10]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.02887,
 'namespaces': {'': {'vector_count': 2887}},
 'total_vector_count': 2887}

In [37]:
j =0
books_names = ['Outlive','The_4_Hour_Body','How_Not_to_Die']

for book in full_books:
    j+=1
    chank_list =[]
    txt_chunks = text_splitter.split_text(book)
    for i in range(len(txt_chunks)):
        chank_list.append({'id':str(j)+str(i),'values':get_embeddings_vector(txt_chunks[i]),'metadata':{'text':txt_chunks[i],'chank':i,'book_name':books_names[j-1]}})
        #בהמשך יש להחליף את הדפים לפונקציה שתזהה מאיזה דף לקחת
        if i%100==0:
            index.upsert(chank_list)
            print(i)
            chank_list=[]
    index.upsert(chank_list)
    

0
100
200
300
400
500
600
700
800
0
100
200
300
400
500
600
700
0
100
200
300
400
500
600
700
800
900
1000
1100
1200


## legacy: prompt - for base rag 

In [12]:
query = "what shuold i eat for long life?"

embed = get_embeddings_vector(query)
res = index.query(vector=embed.tolist(), top_k=3, include_metadata=True)

contexts = [
    x['metadata']['text'] for x in res['matches']
]

prompt_start = (
    '''You are a chat designed to give advice for healthy living.
     The excerpts below are taken from instructional books on how to live a healthy life
    
    Answer the question based on the context below.
    \n\n"+
    "Context:\n
    
    '''
)

prompt_end = (
    f"\n\nQuestion: {query}\nAnswer:"
)

prompt = (
    prompt_start + "\n\n---\n\n".join(contexts) + 
    prompt_end
)

In [ ]:
books_names = ['Outlive','The_4_Hour_Body','How_Not_to_Die']

def query_with_meta(query, book_name:list[str]=None):
    embed = get_embeddings_vector(query)
    #
    if book_name:
        clean_list = []
        for book in book_name:
            if book in books_names: 
                clean_list.append(book)
        if clean_list:
            res = index.query(vector=embed.tolist(), top_k=4,
                            filter={"book_name": {"$in": clean_list}},
                            include_metadata=True)
        else:
            res = index.query(vector=embed.tolist(), top_k=4, include_metadata=True)  
    else: 
        res = index.query(vector=embed.tolist(), top_k=4, include_metadata=True)

    return res